In [8]:
# import modules
import pandas
from sklearn import svm
import tensorflow as tf
import numpy as np
from sklearn.decomposition import PCA
from pylab import *
import struct
import keras as ks
import logging
from keras.layers import Dense, Activation, Flatten, Convolution2D
from keras.utils import np_utils
from keras.models import model_from_json
from keras import backend as K

from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

import matplotlib.pyplot as plt
from skimage import io
import numpy as np
from PIL import Image 
from scipy import misc
import os
# ...

## 数据读取和预处理
label 3个 => 4个
- normal: 2462
- cpu   : 373
- mem   : 266
- io    : 592

In [19]:
# 3888 * 51 with 4 labels
data_raw = pandas.read_csv("data/host10280-labeled-2.csv")
data = np.array(data_raw)
data_raw = pandas.read_csv("data/host10274-labeled-2.csv")
data = np.concatenate((data,np.array(data_raw)),axis=0)
data_raw = pandas.read_csv("data/host10283-labeled-2.csv")
data = np.concatenate((data,np.array(data_raw)),axis=0)

# 50 = 2 ids + 45 features + 3 labels
# ids: host + clock
# labels: normal, cpu, mem, io
data_features = data[:,2:47]
data_labels = data[:,46:50]

len(data_labels)
for i in range(len(data_labels)):
    item = data_labels[i]
    if item[1] == 0 and item[2] == 0 and item[3] == 0:
        data_labels[i][0] = 1
    else:
        data_labels[i][0] = 0

data_class = [] # 0 ~ 3
for label in data_labels:
    data_class.append(np.dot([0,1,2,3],label))
data_class = np.array(data_class,dtype="int")

print(sum(data_class == 0))
print(sum(data_class == 1))
print(sum(data_class == 2))
print(sum(data_class == 3))
print(data_class.shape)

2462
373
266
592
(3693,)


In [10]:
# 1231 abnormal
num_sample = sum(data_class != 0)
# select 1231 normal samples randomly
data_tmp = data_features[data_class == 0]
index_tmp = np.arange(0,data_tmp.shape[0],data_tmp.shape[0]/num_sample)[0:num_sample]
data_normal_tmp = data_tmp[index_tmp]

data_features = np.concatenate((data_normal_tmp,                 data_features[data_class != 0]),axis=0)
data_labels   = np.concatenate((np.array([[1,0,0,0]]*num_sample),data_labels[data_class != 0]),  axis=0)
data_class    = np.concatenate((np.array([0]*num_sample),        data_class[data_class != 0]),   axis=0)

## 调整为2分类问题

In [20]:
data_labels = data_labels[:,0:2]
data_labels[data_class != 0] = [0,1]
data_labels[data_class == 0] = [1,0]
data_class[data_class != 0] = 1
data_class[data_class == 0] = 0

## 划分训练集和验证集
- 训练集: 80%
- 验证集: 20%

In [22]:
# feature scaling
for i in range(data_features.shape[1]):
    d_min = data_features[:,i].min()
    d_max = data_features[:,i].max()
    if d_min == d_max:
        data_features[:,i] = 1
        continue
    data_features[:,i] -= d_min
    data_features[:,i] /= (d_max - d_min)

num_tosample = len(data_class)
index_test = np.arange(0,num_tosample,5)
index_train = np.array(list(set(np.arange(0,num_tosample,1)) - set(index_test)))

data_train = data_features[index_train]
labels_train = data_labels[index_train]
class_train = data_class[index_train]
data_test = data_features[index_test]
labels_test = data_labels[index_test]
class_test = data_class[index_test]
print(data_train.shape)
print(labels_train.shape)
print(class_train.shape)
print(data_test.shape)
print(labels_test.shape)
print(class_test.shape)

(2954, 45)
(2954, 2)
(2954,)
(739, 45)
(739, 2)
(739,)


## 神经网络
45 => 128 => 64 => 32 => 16 => 4

In [15]:
model = ks.models.Sequential()
model.add(Dense(128, input_dim=data_features.shape[1]))
model.add(Activation('relu'))
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dense(32))
model.add(Activation('relu'))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(2))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',optimizer='adadelta',metrics=['accuracy'])
model.fit(x=data_train,y=labels_train,batch_size=128,nb_epoch=1000,verbose=1,validation_data=(data_test,labels_test))

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


/Users/lyq/Anaconda/anaconda2/lib/python2.7/site-packages/keras/models.py:848: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 1969 samples, validate on 493 samples
Epoch 1/1000
1969/1969 [==============================] - 0s - loss: 0.6361 - acc: 0.7369 - val_loss: 0.5274 - val_acc: 0.9432
Epoch 2/1000
1969/1969 [==============================] - 0s - loss: 0.4003 - acc: 0.9528 - val_loss: 0.2418 - val_acc: 1.0000
Epoch 3/1000
1969/1969 [==============================] - 0s - loss: 0.2145 - acc: 0.9619 - val_loss: 0.0865 - val_acc: 1.0000
Epoch 4/1000
1969/1969 [==============================] - 0s - loss: 0.0715 - acc: 0.9995 - val_loss: 0.0336 - val_acc: 1.0000
Epoch 5/1000
1969/1969 [==============================] - 0s - loss: 0.0254 - acc: 1.0000 - val_loss: 0.0189 - val_acc: 1.0000
Epoch 6/1000
1969/1969 [==============================] - 0s - loss: 0.0141 - acc: 1.0000 - val_loss: 0.0104 - val_acc: 1.0000
Epoch 7/1000
1969/1969 [==============================] - 0s - loss: 0.0091 - acc: 1.0000 - val_loss: 0.0072 - val_acc: 1.0000
Epoch 8/1000
1969/1969 [==============================] - 0s - l

1969/1969 [==============================] - 0s - loss: 9.3731e-06 - acc: 1.0000 - val_loss: 8.7334e-06 - val_acc: 1.0000
Epoch 62/1000
1969/1969 [==============================] - 0s - loss: 9.1250e-06 - acc: 1.0000 - val_loss: 8.5103e-06 - val_acc: 1.0000
Epoch 63/1000
1969/1969 [==============================] - 0s - loss: 8.8561e-06 - acc: 1.0000 - val_loss: 8.2662e-06 - val_acc: 1.0000
Epoch 64/1000
1969/1969 [==============================] - 0s - loss: 8.6372e-06 - acc: 1.0000 - val_loss: 8.0659e-06 - val_acc: 1.0000
Epoch 65/1000
1969/1969 [==============================] - 0s - loss: 8.4174e-06 - acc: 1.0000 - val_loss: 7.8596e-06 - val_acc: 1.0000
Epoch 66/1000
1969/1969 [==============================] - 0s - loss: 8.2049e-06 - acc: 1.0000 - val_loss: 7.7335e-06 - val_acc: 1.0000
Epoch 67/1000
1969/1969 [==============================] - 0s - loss: 7.9914e-06 - acc: 1.0000 - val_loss: 7.5017e-06 - val_acc: 1.0000
Epoch 68/1000
1969/1969 [==============================] - 0s 

1969/1969 [==============================] - 0s - loss: 3.2907e-06 - acc: 1.0000 - val_loss: 3.1305e-06 - val_acc: 1.0000
Epoch 122/1000
1969/1969 [==============================] - 0s - loss: 3.2535e-06 - acc: 1.0000 - val_loss: 3.0899e-06 - val_acc: 1.0000
Epoch 123/1000
1969/1969 [==============================] - 0s - loss: 3.2178e-06 - acc: 1.0000 - val_loss: 3.0484e-06 - val_acc: 1.0000
Epoch 124/1000
1969/1969 [==============================] - 0s - loss: 3.1800e-06 - acc: 1.0000 - val_loss: 3.0062e-06 - val_acc: 1.0000
Epoch 125/1000
1969/1969 [==============================] - 0s - loss: 3.1432e-06 - acc: 1.0000 - val_loss: 2.9763e-06 - val_acc: 1.0000
Epoch 126/1000
1969/1969 [==============================] - 0s - loss: 3.1095e-06 - acc: 1.0000 - val_loss: 2.9486e-06 - val_acc: 1.0000
Epoch 127/1000
1969/1969 [==============================] - 0s - loss: 3.0749e-06 - acc: 1.0000 - val_loss: 2.9093e-06 - val_acc: 1.0000
Epoch 128/1000
1969/1969 [==============================

...

1969/1969 [==============================] - 0s - loss: 3.5696e-07 - acc: 1.0000 - val_loss: 3.4602e-07 - val_acc: 1.0000
Epoch 712/1000
1969/1969 [==============================] - 0s - loss: 3.5584e-07 - acc: 1.0000 - val_loss: 3.4481e-07 - val_acc: 1.0000
Epoch 713/1000
1969/1969 [==============================] - 0s - loss: 3.5515e-07 - acc: 1.0000 - val_loss: 3.4433e-07 - val_acc: 1.0000
Epoch 714/1000
1969/1969 [==============================] - 0s - loss: 3.5424e-07 - acc: 1.0000 - val_loss: 3.4360e-07 - val_acc: 1.0000
Epoch 715/1000
1969/1969 [==============================] - 0s - loss: 3.5348e-07 - acc: 1.0000 - val_loss: 3.4336e-07 - val_acc: 1.0000
Epoch 716/1000
1969/1969 [==============================] - 0s - loss: 3.5275e-07 - acc: 1.0000 - val_loss: 3.4288e-07 - val_acc: 1.0000
Epoch 717/1000
1969/1969 [==============================] - 0s - loss: 3.5224e-07 - acc: 1.0000 - val_loss: 3.4239e-07 - val_acc: 1.0000
Epoch 718/1000
1969/1969 [==============================

1969/1969 [==============================] - 0s - loss: 3.2324e-07 - acc: 1.0000 - val_loss: 3.1410e-07 - val_acc: 1.0000
Epoch 771/1000
1969/1969 [==============================] - 0s - loss: 3.2288e-07 - acc: 1.0000 - val_loss: 3.1398e-07 - val_acc: 1.0000
Epoch 772/1000
1969/1969 [==============================] - 0s - loss: 3.2257e-07 - acc: 1.0000 - val_loss: 3.1350e-07 - val_acc: 1.0000
Epoch 773/1000
1969/1969 [==============================] - 0s - loss: 3.2215e-07 - acc: 1.0000 - val_loss: 3.1338e-07 - val_acc: 1.0000
Epoch 774/1000
1969/1969 [==============================] - 0s - loss: 3.2170e-07 - acc: 1.0000 - val_loss: 3.1362e-07 - val_acc: 1.0000
Epoch 775/1000
1969/1969 [==============================] - ETA: 0s - loss: 3.0547e-07 - acc: 1.000 - 0s - loss: 3.2136e-07 - acc: 1.0000 - val_loss: 3.1302e-07 - val_acc: 1.0000
Epoch 776/1000
1969/1969 [==============================] - 0s - loss: 3.2112e-07 - acc: 1.0000 - val_loss: 3.1217e-07 - val_acc: 1.0000
Epoch 777/1000

1969/1969 [==============================] - 0s - loss: 3.0402e-07 - acc: 1.0000 - val_loss: 2.9621e-07 - val_acc: 1.0000
Epoch 830/1000
1969/1969 [==============================] - 0s - loss: 3.0368e-07 - acc: 1.0000 - val_loss: 2.9548e-07 - val_acc: 1.0000
Epoch 831/1000
1969/1969 [==============================] - 0s - loss: 3.0317e-07 - acc: 1.0000 - val_loss: 2.9524e-07 - val_acc: 1.0000
Epoch 832/1000
1969/1969 [==============================] - 0s - loss: 3.0278e-07 - acc: 1.0000 - val_loss: 2.9500e-07 - val_acc: 1.0000
Epoch 833/1000
1969/1969 [==============================] - 0s - loss: 3.0247e-07 - acc: 1.0000 - val_loss: 2.9476e-07 - val_acc: 1.0000
Epoch 834/1000
1969/1969 [==============================] - 0s - loss: 3.0217e-07 - acc: 1.0000 - val_loss: 2.9428e-07 - val_acc: 1.0000
Epoch 835/1000
1969/1969 [==============================] - 0s - loss: 3.0193e-07 - acc: 1.0000 - val_loss: 2.9415e-07 - val_acc: 1.0000
Epoch 836/1000
1969/1969 [==============================

1969/1969 [==============================] - 0s - loss: 2.8710e-07 - acc: 1.0000 - val_loss: 2.8025e-07 - val_acc: 1.0000
Epoch 889/1000
1969/1969 [==============================] - 0s - loss: 2.8676e-07 - acc: 1.0000 - val_loss: 2.8001e-07 - val_acc: 1.0000
Epoch 890/1000
1969/1969 [==============================] - 0s - loss: 2.8646e-07 - acc: 1.0000 - val_loss: 2.7953e-07 - val_acc: 1.0000
Epoch 891/1000
1969/1969 [==============================] - 0s - loss: 2.8619e-07 - acc: 1.0000 - val_loss: 2.7928e-07 - val_acc: 1.0000
Epoch 892/1000
1969/1969 [==============================] - 0s - loss: 2.8610e-07 - acc: 1.0000 - val_loss: 2.7904e-07 - val_acc: 1.0000
Epoch 893/1000
1969/1969 [==============================] - 0s - loss: 2.8591e-07 - acc: 1.0000 - val_loss: 2.7856e-07 - val_acc: 1.0000
Epoch 894/1000
1969/1969 [==============================] - 0s - loss: 2.8558e-07 - acc: 1.0000 - val_loss: 2.7832e-07 - val_acc: 1.0000
Epoch 895/1000
1969/1969 [==============================

1969/1969 [==============================] - 0s - loss: 2.7450e-07 - acc: 1.0000 - val_loss: 2.6768e-07 - val_acc: 1.0000
Epoch 948/1000
1969/1969 [==============================] - ETA: 0s - loss: 2.2165e-07 - acc: 1.000 - 0s - loss: 2.7435e-07 - acc: 1.0000 - val_loss: 2.6768e-07 - val_acc: 1.0000
Epoch 949/1000
1969/1969 [==============================] - 0s - loss: 2.7399e-07 - acc: 1.0000 - val_loss: 2.6731e-07 - val_acc: 1.0000
Epoch 950/1000
1969/1969 [==============================] - 0s - loss: 2.7372e-07 - acc: 1.0000 - val_loss: 2.6707e-07 - val_acc: 1.0000
Epoch 951/1000
1969/1969 [==============================] - 0s - loss: 2.7353e-07 - acc: 1.0000 - val_loss: 2.6707e-07 - val_acc: 1.0000
Epoch 952/1000
1969/1969 [==============================] - 0s - loss: 2.7338e-07 - acc: 1.0000 - val_loss: 2.6671e-07 - val_acc: 1.0000
Epoch 953/1000
1969/1969 [==============================] - 0s - loss: 2.7323e-07 - acc: 1.0000 - val_loss: 2.6671e-07 - val_acc: 1.0000
Epoch 954/1000

## 神经网络 + SVM
45 => 128 => 64 => 32 => 16 => SVM

In [16]:
get_feature = K.function([model.layers[0].input],[model.layers[2].output])
mid_features_train = get_feature([data_train])[0]
mid_features_test  = get_feature([data_test])[0]
clf = svm.SVC()
clf.fit(mid_features_train, class_train)
print(clf.score(mid_features_train, class_train))
print(clf.score(mid_features_test, class_test))

1.0
1.0


## 其他机器学习算法
- KNN
- 线性SVM
- 基于核函数的SVM (RBF SVM), Radial basis function kernel
- 决策树
- 随机森林

In [17]:
classifiers = [
    KNeighborsClassifier(4),
    svm.SVC(kernel="linear", C=0.025),
    svm.SVC(),
    DecisionTreeClassifier(max_depth=3),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1)
    ]
names = ["Nearest Neighbors", "Linear SVM", "RBF SVM", "Decision Tree",
         "Random Forest"]
for name, clf in zip(names,classifiers):
    clf.fit(data_train,class_train)
    print(name)
    print("train: {}".format(clf.score(data_train, class_train)))
    print("test: {}".format(clf.score(data_test, class_test)))

Nearest Neighbors
train: 1.0
test: 1.0
Linear SVM
train: 1.0
test: 1.0
RBF SVM
train: 1.0
test: 1.0
Decision Tree
train: 1.0
test: 1.0
Random Forest
train: 0.939563230066
test: 0.941176470588
